# Descripción

Este notebook contiene los pasos requeridos para asignar las combinaciones, previamente asignadas, profesor-curso a un salón de acuerdo a los requerimientos institucionales.

In [1]:
import json

# READ DATA

## first stage model output

In [2]:
# basic parameters
DAY_LENGTH=16

# Horizonte de planificación
HORIZON=5

#  Días de la semana
DAYS=['Lunes','Martes','Miercoles','Jueves','Viernes']


In [3]:
# Este diccionario contiene un mapa de los slots de tiempo a los días y horas

actual_time={}
for d in range(HORIZON):
    for h in range(DAY_LENGTH):
        actual_time[d*16+h]={"slot":DAYS[d]+' '+str(h+6)+':00', "day":d, "hour":h,'slot_abr':DAYS[d][0]+str(h)}

In [4]:
# read model_output.json
allocations=json.load(open('data/model_output.json','r'))
#allocations

In [5]:
groups=allocations['courses']
teachers=allocations['teachers']
pairs=allocations['ensembles']
teachers=set([int(t) for t in teachers])
groups=set([g for g in groups])
pairs=[(int(p[0]) ,p[1]) for p in pairs]

In [6]:
courses_by_teacher={}
for p in pairs:
    if p[0] not in courses_by_teacher:
        courses_by_teacher[p[0]]=[p[1]]
    else:
        courses_by_teacher[p[0]].append(p[1])


## Initial data load

In [8]:
# read data.json
horas_curso=json.load(open('data/horas_curso.json','r'))
grupo_requerimiento=json.load(open('data/grupo_requerimiento.json','r'))
ubicacion_semestral=json.load(open('data/ubicacion_semestral.json','r'))

In [63]:
tipo_salon={'A','B','C','D','E','F','G'}
hora_disponibilidad_x_salon={
    'A':10
    ,'B':0
    ,'C':0
    ,'D':10
    ,'E':10
    ,'F':10
    ,'G':10
}

cantidad_de_salones={
 'A':11
 ,'B':9
 ,'C':3
 ,'D':1
 ,'E':1
 ,'F':2
 ,'G':1   
}

# Modelo

In [65]:
I=set(teachers)
J=set(groups)
K=set(pairs)
C= tipo_salon

In [42]:
from itertools import product

# courses that are the same
same_course={}
# courses that are taught by the same teacher
same_teacher={}

# delta semestres
delta_semestre={}
for j1,j2 in product(J,J):
    same_course[(j1,j2)]=0
    same_teacher[(j1,j2)]=0
    if j1[:9]==j2[:9]:
        same_course[(j1,j2)]=1
    delta_semestre[(j1,j2)]=abs(ubicacion_semestral[j1[:9]]["Semestre"]-ubicacion_semestral[j2[:9]]["Semestre"])

for p in pairs:
    for p2 in pairs:
        if p[0]==p2[0]:
            same_teacher[(p[1],p2[1])]=1

# course and time slot compatibility
time_compatibility={}
for c,j in product(C,J):
    for s in actual_time.keys():
        time_compatibility[(c,j,s)]=0
        if actual_time[s]['hour'] + horas_curso[j[:9]]['horas']-1<DAY_LENGTH:
            time_compatibility[(c,j,s)]=1

valid_keys=[]
for c,j,s in product(C,J,actual_time.keys()):
    if time_compatibility[(c,j,s)]==1:
        valid_keys.append((c,j,s))


In [53]:
valid_keys

[('F', '9937-0014-0', 0),
 ('F', '9937-0014-0', 1),
 ('F', '9937-0014-0', 2),
 ('F', '9937-0014-0', 3),
 ('F', '9937-0014-0', 4),
 ('F', '9937-0014-0', 5),
 ('F', '9937-0014-0', 6),
 ('F', '9937-0014-0', 7),
 ('F', '9937-0014-0', 8),
 ('F', '9937-0014-0', 9),
 ('F', '9937-0014-0', 10),
 ('F', '9937-0014-0', 11),
 ('F', '9937-0014-0', 12),
 ('F', '9937-0014-0', 16),
 ('F', '9937-0014-0', 17),
 ('F', '9937-0014-0', 18),
 ('F', '9937-0014-0', 19),
 ('F', '9937-0014-0', 20),
 ('F', '9937-0014-0', 21),
 ('F', '9937-0014-0', 22),
 ('F', '9937-0014-0', 23),
 ('F', '9937-0014-0', 24),
 ('F', '9937-0014-0', 25),
 ('F', '9937-0014-0', 26),
 ('F', '9937-0014-0', 27),
 ('F', '9937-0014-0', 28),
 ('F', '9937-0014-0', 32),
 ('F', '9937-0014-0', 33),
 ('F', '9937-0014-0', 34),
 ('F', '9937-0014-0', 35),
 ('F', '9937-0014-0', 36),
 ('F', '9937-0014-0', 37),
 ('F', '9937-0014-0', 38),
 ('F', '9937-0014-0', 39),
 ('F', '9937-0014-0', 40),
 ('F', '9937-0014-0', 41),
 ('F', '9937-0014-0', 42),
 ('F', '993

In [48]:
import gurobipy as gp

# declare an empty model
m=gp.Model('Asignacion de salones')
m.setParam('OutputFlag',False)

# model sense
m.modelSense=gp.GRB.MAXIMIZE

# declares variables

# y representa si el cursu j se asigna a un salón tipo c iniciando en el slot s
y=m.addVars(valid_keys,vtype=gp.GRB.BINARY,name='y')
# x representa si el curso j ocupa un salón tipo c durante el slot s
x=m.addVars([(c,j,s) for c in C for j in J for s in actual_time.keys()],vtype=gp.GRB.BINARY,name='x')
# indica si dos cursos se solapan en algún momento
z=m.addVars([(j1,j2) for j1,j2 in product(J,J) if j1!=j2],vtype=gp.GRB.BINARY,name='z')

El objetivo es generar una programación tal que los cursos de semestres similares se
programan en diferentes momentos del día. Es decir, queremos maximizar la diferencia de semestre entre los cursos que se deban programar a la misma hora.

$ \max:\sum_{{j1,j2 \in J, j1 \neq j2}} z[j1,j2] \cdot \delta_{\text{{semestre}}}[j1,j2]$


In [49]:
# funcion objetivo:
# maximizar la suma de los deltas semestrales de los cursos que se imparten
m.setObjective(gp.quicksum(z[j1,j2]*delta_semestre[j1,j2] for j1,j2 in product(J,J) if j1!=j2))

Restricción 1: Todos los cursos deben tener exactamente una asignación de inicio.

$\sum_{c,j1,s \in \text{{valid\_keys}}, j1=j} y[(c,j1,s)] = 1 \quad \forall j \in J$


In [50]:
# each course starts at one time slot
m.addConstrs((gp.quicksum(y[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==1 for j in J),name='each_course_starts_at_one_time_slot')
0

0

Restricción 2:

si un curso da inicio durante el slot s en el salón tipo c, y tiene una duración t, ese curso debe estar ocupando un slot entre s, s+1 hasta s+t-1

$  
\sum_{s1 \in \text{{range}}(s,s+\text{{horas\_curso}}[j1[:9]]['horas']) \atop (c,j1,s1) \in \text{{valid\_keys}}} x[(c,j1,s1)] \geq y[(c,j1,s)] \cdot \text{{horas\_curso}}[j1[:9]]['horas'] \forall c, j1, s \in \text{{valid\_keys}}
$

In [30]:
print(horas_curso)

{'8830-0001': {'ASIGNATURA': 'Introducción_Ingeniería_Industrial', 'codigo': '8830-0001', 'horas': 2, 'cantidad_de_cursos': 1, 'total_estudiantes': 20, 'horas_no_instruccion': 0.75}, '2027-0059': {'ASIGNATURA': 'Fundamentos_dibujo_CAD', 'codigo': '2027-0059', 'horas': 2, 'cantidad_de_cursos': 2, 'total_estudiantes': 14, 'horas_no_instruccion': 0.0}, '9937-0021': {'ASIGNATURA': 'OCBD_Matematica_Operativa', 'codigo': '9937-0021', 'horas': 3, 'cantidad_de_cursos': 1, 'total_estudiantes': 18, 'horas_no_instruccion': 0.75}, '9937-0062': {'ASIGNATURA': 'Calculo_Diferencial', 'codigo': '9937-0062', 'horas': 4, 'cantidad_de_cursos': 1, 'total_estudiantes': 20, 'horas_no_instruccion': 0.75}, '9937-0063': {'ASIGNATURA': 'Geometría_Analítica', 'codigo': '9937-0063', 'horas': 4, 'cantidad_de_cursos': 1, 'total_estudiantes': 22, 'horas_no_instruccion': 0.75}, '6611-0004': {'ASIGNATURA': 'Economía', 'codigo': '6611-0004', 'horas': 2, 'cantidad_de_cursos': 2, 'total_estudiantes': 14, 'horas_no_instru

In [55]:
"""cont=0
for c,j1,s in valid_keys:
    lhs=gp.quicksum(x[(c,j1,s1)] for s1 in range(s,s+horas_curso[j1[:9]]['horas']) )
    rhs=y[(c,j1,s)]*horas_curso[j1[:9]]['horas']
    print(lhs,">=",rhs)
    cont+=1
    if cont==16:
        break"""
    

m.addConstrs((gp.quicksum(x[(c,j1,s1)] for s1 in range(s,s+horas_curso[j1[:9]]['horas'])  )
             >=y[(c,j1,s)]*horas_curso[j1[:9]]['horas'] for c,j1,s in valid_keys))
0

0

Restricción 3:
Cada curso es asignado un bloque de tiempo equivalente a las horas que requiere


$ \sum_{c,j1,s \in \text{{valid\_keys}} \atop j1=j} y[(c,j1,s)] = \text{{horas\_curso}}[j[:9]]['horas'] \forall j \in J$



In [57]:
m.addConstrs((gp.quicksum(x[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==horas_curso[j[:9]]['horas'] for j in J),name='each_course_is_assigned_the_time_it_requires')
0

0

$\forall j1, j2 \in J \quad \text{{s.t.}} \quad j1 \neq j2, \forall c1, c2 \in C, \forall s \in \text{{actual\_time.keys()}}, z[j1,j2] \geq x[c1,j1,s] + x[c2,j2,s] - 1$


In [59]:
m.addConstrs((z[j1,j2]>=x[c1,j1,s]+x[c2,j2,s]-1 for j1,j2 in product(J,J) if j1!=j2 for c1 ,c2 in product(C,C) for s in actual_time.keys()),name='si_dos_cursos_se_imparten_a_la_misma_hora_en_cualquier_salon_se_debe_activar_la_variable_z_que_lleva_cuentas_de_los_solapamientos')
0

0

In [66]:

m.addConstrs((gp.quicksum(x[(c,j,s)] for j in J )<=cantidad_de_salones[c]  for c in C for s in actual_time.keys()),name='cada_curso_se_imparte_en_un_maximo_de_un_salon_de_cada_tipo')
0


0

In [67]:
time_limit=60
m.setParam('TimeLimit',time_limit)
m.optimize()

# print model status
if m.status==gp.GRB.OPTIMAL:
    print('Optimal solution found with objective: %g' % m.objVal)
elif m.status==gp.GRB.TIME_LIMIT:
    print('Time limit reached with objective: %g' % m.objVal)
else:
    print('Model is infeasible')

Optimal solution found with objective: 10382


In [69]:
print('Time elapsed: %g' % m.Runtime)
# print the keys for the y variables that are equal to 1
for c,j,s in valid_keys:
    if y[c,j,s].x>=0.9:
        print(c,j,s)

Time elapsed: 12.7034
F 8831-0028-0 71
F 9937-0021-0 45
F 8827-0008-0 41
F 9937-0018-0 1
F 6611-0004-1 22
F 2027-0059-1 76
F 6069-0018-0 18
F 8830-0001-0 21
F 9937-0074-0 12
E 9937-0076-0 60
E 6069-0014-0 44
E 6069-2060-0 5
E 8833-0021-0 12
E 6611-0004-0 28
E 9939-0018-0 55
A 5555-0010-0 48
A 9937-0073-0 7
A 6069-0127-0 72
A 9935-0031-0 32
A 9937-0011-0 20
A 6069-0012-0 16
A 9937-0012-0 38
A 8830-0009-0 67
A 2027-0059-0 59
A 9937-0062-0 74
A 5559-0014-0 3
G 9937-0014-0 25
G 9935-0032-0 20
G 9935-0012-0 13
G 5569-0011-1 77
G 9937-0072-0 61
G 8830-0010-0 68
D 6069-0154-0 49
D 8833-0083-0 17
C 6612-0001-0 4
C 8830-0008-0 55
C 6069-0175-0 7
C 5566-0009-0 48
C 5569-0011-0 27
C 8830-0007-0 12
B 8830-0006-0 40
B 5569-0011-2 53
B 6069-0011-0 45
B 6069-0078-0 66
B 8827-0120-0 60
B 9937-0062-1 35
B 6611-0088-0 33
B 9937-0063-0 38
B 9937-0013-0 58
B 8830-0012-0 54
B 8828-0028-0 48
H 8830-0005-0 4
H 8830-0004-0 2
H 8830-0011-0 58
H 8821-0010-0 37
H 8821-0002-0 10
H CEEM-0014-0 29
H 9935-0033-0 49
